In [2]:
#Import packages
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [3]:
image = input("Name of the picture you want to see (complete path): ")
img = cv2.imread(str(image))
print("Please close image window to proceed with the code")
# cv2.imshow("image",img);
# cv2.waitKey(0)
#G:\My Drive\24-678 Computer Vision for Engineers\Project\osteoporosis\6.png

Name of the picture you want to see (complete path): G:\My Drive\24-678 Computer Vision for Engineers\Project\osteoporosis\6.png
Please close image window to proceed with the code


In [4]:
img_g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [5]:
def Canny_edge(image_name,img,k_size=5,l_limit=10,u_limit=110):
    blur = cv2.GaussianBlur(img,(k_size,k_size),cv2.BORDER_DEFAULT)
    edges = cv2.Canny(blur,l_limit,u_limit)
    cv2.namedWindow("Canny",cv2.WINDOW_NORMAL)
    cv2.imshow("Canny",edges)
    cv2.waitKey(0)
    save = input("Save the image? ")#(press y to save image)
    if save == "Y" or save == "y":
        cv2.imwrite(image_name.split("\\")[-1][:-4]+"_edge.png",edges)
    #cv2.waitKey(0)
    return edges

In [6]:
Canny_edge(image,img_g)

Save the image? n


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [255, 255, 255, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [7]:
def pseudo(image_name,image):
    img2 = image.copy().astype("float32")
    for i in range(img2.shape[2]):
        img2[:,:,i] = (img2[:,:,i]-np.min(img2[:,:,i]))/(np.max(img2[:,:,i])-np.min(img2[:,:,i]))
        img2[:,:,i] *= 255

    lut_list_bgr = np.zeros((256,1,3)).astype("uint8")

    for i in range(128,192):
        lut_list_bgr[i,0,2] = 4*(i-127.5)
        lut_list_bgr[i,0,1] = 255

    for j in range(192,256):
        lut_list_bgr[j,0,2] = 255
        lut_list_bgr[j,0,1] = 4*(191.25-j) + 255

    for k in range(0,64):
        lut_list_bgr[k,0,0] = 255
        lut_list_bgr[k,0,1] = 4*k

    for m in range(64,128):
        lut_list_bgr[m,0,0] = 4*(63.75-m)+255
        lut_list_bgr[m,0,1] = 255

    lut_list_bgrop = lut_list_bgr.astype("uint8")
    img2 = img2.astype("uint8")

    imp = cv2.LUT(img2,lut_list_bgrop)
    
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    centre_coord = np.argwhere(img_gray==np.amax(img_gray))
    centre = (np.mean(centre_coord,axis=0)).astype(int)
    count = 0
    for i in centre_coord:
        imp = cv2.circle(imp,(i[1],i[0]),15,(255,255,255),2)
        count += 1
#     imp = cv2.line(imp,(centre[1]-25,centre[0]),(centre[1]+25,centre[0]),(255,255,255),2)
#     imp = cv2.line(imp,(centre[1],centre[0]-25),(centre[1],centre[0]+25),(255,255,255),2)
    
    cv2.imshow("applyColorMap",imp)
    cv2.waitKey(0)
    save = input("Save the image? ")#(press y to save image)
    if save == "Y" or save == "y":
        cv2.imwrite(image_name.split("\\")[-1][:-4]+"_pseudo.png",imp)
    #cv2.waitKey(0)
    return count, imp

In [8]:
pseudo(image,img)

Save the image? n


(3604,
 array([[[255,   0,   0],
         [255,   0,   0],
         [255,   0,   0],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255,   0,   0],
         [255,   0,   0],
         [255,   0,   0],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255,   0,   0],
         [255,   0,   0],
         [255,   0,   0],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 2

In [228]:
def bounding_box(image_name,img):
    copy = img.copy()
    #copy = cv2.GaussianBlur(copy,(3,3),cv2.BORDER_DEFAULT)
    gray = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray,127, 255, cv2.THRESH_BINARY_INV)[1]
    
#     # clean up
#     for i in range(1):
#         thresh = cv2.erode(thresh, None)
#     for i in range(1):
#         thresh = cv2.dilate(thresh, None)

    cnts,hier = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    length_arr = []
    
    for i in range(0, len(cnts)):
        length = cv2.arcLength(cnts[i], False)
        length_arr.append(length)
        if length>80 and length<6000:
            x,y,w,h = cv2.boundingRect(cnts[i])
            cv2.rectangle(copy,(x,y),(x+w,y+h),(0,0,255),2)

    copy[0,:] = np.zeros(copy[0,:].shape)
    copy[1,:] = np.zeros(copy[1,:].shape)
    copy[-1,:] = np.zeros(copy[-1,:].shape)
    copy[-2,:] = np.zeros(copy[-2,:].shape)
    copy[:,0] = np.zeros(copy[:,0].shape)
    copy[:,1] = np.zeros(copy[:,1].shape)
    
    for q in range(1,6):
        copy[:,-q] = np.zeros(copy[:,-q].shape)
    
    cv2.imshow('thresh', copy)
    cv2.waitKey(0)
    return cnts

In [229]:
bounding_box(image,img)

[array([[[470, 913]]], dtype=int32),
 array([[[471, 889]]], dtype=int32),
 array([[[470, 882]]], dtype=int32),
 array([[[469, 879]]], dtype=int32),
 array([[[469, 873]]], dtype=int32),
 array([[[471, 864]]], dtype=int32),
 array([[[470, 846]]], dtype=int32),
 array([[[473, 845]]], dtype=int32),
 array([[[472, 838]],
 
        [[473, 838]]], dtype=int32),
 array([[[470, 837]]], dtype=int32),
 array([[[473, 836]]], dtype=int32),
 array([[[472, 792]]], dtype=int32),
 array([[[470, 781]]], dtype=int32),
 array([[[470, 769]],
 
        [[470, 770]],
 
        [[471, 769]]], dtype=int32),
 array([[[468, 769]]], dtype=int32),
 array([[[470, 767]],
 
        [[471, 767]]], dtype=int32),
 array([[[469, 765]]], dtype=int32),
 array([[[472, 763]]], dtype=int32),
 array([[[465, 758]]], dtype=int32),
 array([[[468, 747]]], dtype=int32),
 array([[[465, 742]]], dtype=int32),
 array([[[465, 737]]], dtype=int32),
 array([[[462, 737]]], dtype=int32),
 array([[[464, 734]]], dtype=int32),
 array([[[462, 7